In [ ]:
import pandas as pd

import spacy
from spacy.training.example import Example

# Load train data
train_data = pd.read_csv('/home/rgukt/Downloads/NER-datasets-dcb6c7439a7de43abc2448bad5b1d81a47f26c0d/CONLL2003/train.txt', sep='\t', header=None, names=['Text'])
# You might need to adjust the separator or header as per your file structure

# Load test data
test_data = pd.read_csv('/home/rgukt/Downloads/NER-datasets-dcb6c7439a7de43abc2448bad5b1d81a47f26c0d/CONLL2003/test.txt', sep='\t', header=None, names=['Text'])

# Load validation data
valid_data = pd.read_csv('/home/rgukt/Downloads/NER-datasets-dcb6c7439a7de43abc2448bad5b1d81a47f26c0d/CONLL2003/valid.txt', sep='\t', header=None, names=['Text'])
print(train_data)

In [ ]:
print(test_data)


In [ ]:
print(valid_data)

In [ ]:
import os

# Function to extract words with named entities
def extract_named_entities(file_path):
    named_entities = []
    with open(file_path, 'r') as file:
        sentence = []
        for line in file:
            if line.strip():  # Non-empty line
                splits = line.strip().split(' ')
                word = splits[0]
                entity = splits[-1]
                sentence.append([word, entity])
            else:  # Empty line denotes end of a sentence
                named_entities.append(sentence)
                sentence = []
        if sentence:  # Append any remaining sentence
            named_entities.append(sentence)
    return named_entities

# Path to the 'data' folder containing train.txt, valid.txt, test.txt
data_folder = '/home/rgukt/Downloads/NER-datasets-dcb6c7439a7de43abc2448bad5b1d81a47f26c0d/CONLL2003'

train_named_entities = extract_named_entities(os.path.join(data_folder, 'train.txt'))
valid_named_entities = extract_named_entities(os.path.join(data_folder, 'valid.txt'))
test_named_entities = extract_named_entities(os.path.join(data_folder, 'test.txt'))

# Displaying the extracted words along with their named entities for the first sentence in train data
print(train_named_entities[0])


# Displaying the extracted words along with their named entities for the first sentence in train data
print(test_named_entities[9])


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np 
# Sample function to load and process CoNLL-03 data
def load_conll_data(file_path):
    sentences = []  # List to store sentences
    ner_tags = []   # List to store corresponding NER tags

    with open(file_path, 'r') as file:
        sentence = []
        tags = []
        for line in file:
            if line != '\n':  # Non-empty line
                word, pos, chunk, tag = line.strip().split()  # Assuming the CoNLL-03 format: word POS-tag Chunk-tag NER-tag
                sentence.append(word)
                tags.append(tag)
            else:  # Empty line indicates the end of a sentence
                sentences.append(sentence)
                ner_tags.append(tags)
                sentence = []
                tags = []
        
        # If the file doesn't end with an empty line
        if sentence:
            sentences.append(sentence)
            ner_tags.append(tags)

    return sentences, ner_tags

# Replace this with code to load and preprocess CoNLL-03 dataset
# You'll need to load sentences and their corresponding NER tags

# Example code (this might need modification based on the dataset format)
X_train, y_train = load_conll_data("/home/rgukt/Downloads/NER-datasets-dcb6c7439a7de43abc2448bad5b1d81a47f26c0d/CONLL2003/train.txt")
    
X_test, y_test = load_conll_data("/home/rgukt/Downloads/NER-datasets-dcb6c7439a7de43abc2448bad5b1d81a47f26c0d/CONLL2003/test.txt") 

# Convert words to indices
words = set(word for sentence in X_train for word in sentence)
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx['PAD'] = 0
word2idx['UNK'] = 1
idx2word = {i: w for w, i in word2idx.items()}

# Convert NER tags to indices
tags = set(tag for tags in y_train for tag in tags)
tag2idx = {t: i + 1 for i, t in enumerate(tags)}
tag2idx['PAD'] = 0
idx2tag = {i: t for t, i in tag2idx.items()}

max_len = 50  # Define maximum sequence length

def convert_data_to_index(X, y, data2idx, label2idx, max_len):
    X_idx = [[data2idx.get(w, data2idx['UNK']) for w in s] for s in X]
    y_idx = [[label2idx[t] for t in s] for s in y]
    X_padded = pad_sequences(X_idx, maxlen=max_len, padding='post')
    y_padded = pad_sequences(y_idx, maxlen=max_len, padding='post')
    y_categorical = [to_categorical(i, num_classes=len(label2idx)) for i in y_padded]
    return X_padded, y_categorical

X_train_idx, y_train_cat = convert_data_to_index(X_train, y_train, word2idx, tag2idx, max_len)
X_test_idx, y_test_cat = convert_data_to_index(X_test, y_test, word2idx, tag2idx, max_len)

# Define the NER model
model = Sequential()
model.add(Embedding(len(word2idx), 50, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(units=100, return_sequences=True)))
model.add(Dropout(0.1))
model.add(TimeDistributed(Dense(len(tag2idx), activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_idx, np.array(y_train_cat), batch_size=32, epochs=4, validation_split=0.1)
model.save('ner_model.h5')

# Evaluate the model
loss, accuracy = model.evaluate(X_test_idx, np.array(y_test_cat))
print(f'Loss: {loss}, Accuracy: {accuracy}')



Epoch 1/4
422/422 [==============================] - 46s 92ms/step - loss: 0.6162 - accuracy: 0.8484 - val_loss: 0.2981 - val_accuracy: 0.9115
Epoch 2/4
422/422 [==============================] - 36s 86ms/step - loss: 0.1977 - accuracy: 0.9398 - val_loss: 0.2155 - val_accuracy: 0.9416
Epoch 3/4
422/422 [==============================] - 36s 85ms/step - loss: 0.0800 - accuracy: 0.9798 - val_loss: 0.1913 - val_accuracy: 0.9546
Epoch 4/4
422/422 [==============================] - 36s 86ms/step - loss: 0.0411 - accuracy: 0.9905 - val_loss: 0.1807 - val_accuracy: 0.9569


/home/rgukt/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


116/116 [==============================] - 3s 24ms/step - loss: 0.3031 - accuracy: 0.9308
Loss: 0.30310767889022827, Accuracy: 0.9308136701583862


In [3]:
#task2
from conllu import parse
import tensorflow as tf

# Sample function to load and process CoNLL-U data for POS tagging
def load_conllu_data(file_path):
    sentences = []  # List to store sentences
    pos_tags = []   # List to store corresponding POS tags

    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()
        parsed_data = parse(data)

        for sentence in parsed_data:
            words = []
            tags = []
            for token in sentence:
                words.append(token['form'])
                tags.append(token['upos'])

            sentences.append(words)
            pos_tags.append(tags)

    return sentences, pos_tags

# Load and preprocess the English PUD dataset
X, y = load_conllu_data("/home/rgukt/Downloads/Universal Dependencies 2.12/ud-treebanks-v2.12/UD_English-PUD/en_pud-ud-test.conllu")

# Convert words to indices
word_set = set(word for sentence in X for word in sentence)
word2idx = {word: idx + 1 for idx, word in enumerate(word_set)}
word2idx['PAD'] = 0
idx2word = {idx: word for word, idx in word2idx.items()}

# Convert POS tags to indices
pos_set = set(tag for tags in y for tag in tags)
pos2idx = {tag: idx for idx, tag in enumerate(pos_set)}

# Convert sentences and POS tags to indices
X_idx = [[word2idx[word] for word in sentence] for sentence in X]
y_idx = [[pos2idx[tag] for tag in tags] for tags in y]

# Define the POS tagging model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(word2idx), 32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(pos2idx), activation='softmax'))
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Padding sequences
X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_idx, padding='post')
y_padded = tf.keras.preprocessing.sequence.pad_sequences(y_idx, padding='post')

# Train the model
model.fit(X_padded, y_padded, epochs=10, batch_size=32)


Epoch 1/10
32/32 [==============================] - 5s 48ms/step - loss: 1.9377 - accuracy: 0.6158
Epoch 2/10
32/32 [==============================] - 2s 47ms/step - loss: 1.0719 - accuracy: 0.6653
Epoch 3/10
32/32 [==============================] - 2s 47ms/step - loss: 0.9538 - accuracy: 0.6883
Epoch 4/10
32/32 [==============================] - 1s 44ms/step - loss: 0.9038 - accuracy: 0.7033
Epoch 5/10
32/32 [==============================] - 1s 41ms/step - loss: 0.8811 - accuracy: 0.7048
Epoch 6/10
32/32 [==============================] - 1s 42ms/step - loss: 0.8643 - accuracy: 0.7067
Epoch 7/10
32/32 [==============================] - 1s 43ms/step - loss: 0.8483 - accuracy: 0.7101
Epoch 8/10
32/32 [==============================] - 1s 42ms/step - loss: 0.8261 - accuracy: 0.7297
Epoch 9/10
32/32 [==============================] - 1s 42ms/step - loss: 0.7914 - accuracy: 0.7542
Epoch 10/10
32/32 [==============================] - 1s 42ms/step - loss: 0.7347 - accuracy: 0.7826


In [9]:
# Load and preprocess the test data using the same approach as training data
X_test, y_test = load_conllu_data("/home/rgukt/Downloads/Universal Dependencies 2.12/ud-treebanks-v2.12/UD_English-PUD/en_pud-ud-test.conllu")
# Convert the test data to indices
X_test_idx = [[word2idx.get(word, 0) for word in sentence] for sentence in X_test]
y_test_idx = [[pos2idx[tag] for tag in tags] for tags in y_test]

# Padding sequences for the test data
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_idx, padding='post')
y_test_padded = tf.keras.preprocessing.sequence.pad_sequences(y_test_idx, padding='post')

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_padded, y_test_padded)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')






32/32 [==============================] - 2s 21ms/step - loss: 0.6942 - accuracy: 0.8114
Test Loss: 0.6941821575164795, Test Accuracy: 0.8113728761672974
